In [1]:
def clean_text(text: str) -> str:
    text = text.replace("\n", " ")
    text = text.replace("[ [ [", "[[[").replace("] ] ]", "]]]")
    return text

In [ ]:
from make_eng_graphs import get_syntactic_relations
from nltk.stem import WordNetLemmatizer
import spacy

nlp = spacy.load("en_core_web_lg")
lemmatizer = WordNetLemmatizer()

def get_triplets(text: str) -> list[str]:
    doc = nlp(text)
    triplets = get_syntactic_relations(doc, lemmatizer)
    return triplets

/Users/dmitry/Desktop/Projects/concept-tree/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import re

def clean_word(word: str) -> str:
    reference = False
    if word.startswith("[ [ [ "):
        reference = True
    word = word.replace("[", "").replace("]", "")
    word = word.replace("(", "").replace(")", "")
    word = word.replace(",", "").replace(";", "")
    word = word.replace("-", "").replace(".", "")
    word = word.replace(":", "").replace(".", "")
    word = word.replace("‘", "").replace("’", "")
    word = word.strip()
    word = re.sub(r'\s{2,}', ' ', word).strip()
    if reference:
        word = "[[[" + word + "]]]"
    return word

def clean_triplet(triplet: tuple[str]) -> tuple[str]:
    triplet = (clean_word(word) for word in triplet)
    return triplet

In [4]:
import csv

def process_file(input_file: str, output_file: str):
    with open(input_file, 'r') as f:
        text = f.read()

    text = clean_text(text)
    triplets = get_triplets(text)

    writer = csv.writer(open(output_file, 'w'), delimiter=",")
    for triplet in triplets:
        triplet = clean_triplet(triplet)
        writer.writerow(triplet)

In [ ]:
import os
from pathlib import Path

input_dir_prefix = '/home/kdemyokhin_1/concept-tree-course-work/articles_anaphora_resolved/arxiv-txt-cs'
output_dir_prefix = '/home/kdemyokhin_1/concept-tree-course-work/articles_triples/arxiv-txt-cs'

input_directory = Path(input_dir_prefix)

# Получаем список всех txt файлов рекурсивно (включая поддиректории)
input_files = list(input_directory.rglob("*.txt"))


output_files = [input_file.replace(input_dir_prefix, output_dir_prefix) for input_file in input_files]


In [ ]:
import tqdm

for input_path, output_path in tqdm.tqdm(zip(input_files, output_files)):
    if os.path.exists(output_path):
        continue
    process_file(input_path, output_path)

102it [01:42,  1.01s/it]
